In [76]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
import seaborn as sns
import seaborn.objects as so
import math
from statsmodels.stats import diagnostic
import statsmodels.stats as st
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import statsmodels.regression.recursive_ls as rls
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import pickle

import plotly.express as px

%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [77]:
import plotly.io as pio
import template_plotly
%load_ext autoreload
%autoreload 2

pd.options.plotting.backend = "plotly"

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


In [78]:
df_movie = pd.read_pickle('../data/df_movie.pkl')
df_pairs = pd.read_pickle('../data/df_pairs.pkl')
display(df_movie.sample(2))
display(df_pairs.sample(2))

,Movie_name,Movie_release,Movie_revenue,Movie_runtime,Movie_languages,Movie_countries,Main_genre,Sec_Genre,Movie_rating,Producer,...,Actor_name,Actor_age_release,Inflation Factor for 2023,2023 valued revenue,Avg_revenue_per_film_at_release,Longevity,Number_of_film_at_release,Avg_rating_per_film_at_release,First_film,Actor_main_genre
287768,Midway,1976.0,43220000.0,132.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}",Action/Adventure,Action,6.7,Walter Mirisch,...,Cliff Robertson,52.0,5.41,2.338202e+08,1.533683e+07,20.0,7,6.842857,False,Thriller
43990,Jack,1996.0,58586889.0,113.0,"{""/m/02h40lc"": ""English Language""}","{""/m/09c7w0"": ""United States of America""}",Drama,Science Fiction,5.8,Fred Fuchs,...,Jennifer Lopez,27.0,1.96,1.148303e+08,4.527871e+07,9.0,3,6.550000,False,Romantic comedy


,Movie_name,Movie_release,Actor_pairs,Movie_revenue,Movie_rating,Actor1,Actor2,Age_difference,Film_count_difference,Average_revenue_difference,First_film,First_film_for_one,Number_of_films_together,Same_genre,Genre
119011,Did You Hear About the Morgans?,2009.0,"(David Call, Mario D'Leon)",85280250.0,4.9,David Call,Mario D'Leon,NaN,0,0.000000e+00,True,True,0,True,Romantic comedy
285598,Ocean's Twelve,2004.0,"(Bruce Willis, Don Cheadle)",362744280.0,6.5,Bruce Willis,Don Cheadle,9.0,21,2.266748e+07,False,False,0,False,None


### RQ1: How does the frequency of actor co-starring impact the chemistry and success of collaborative films?

In [79]:
df_pairs.sort_values(by=['Movie_rating', 'Movie_revenue'], ascending=[False, False], inplace=True)

# Rank the movies based on the sorted order
df_pairs['Rank'] = range(1, len(df_pairs) + 1)

# Calculate the ranking ratio
df_pairs['Ranking_ratio'] = 1 - ((df_pairs['Rank'] - 1) / (len(df_pairs) - 1))

In [81]:
df_pairs.sort_values(['Number_of_films_together', 'Ranking_ratio'], ascending= [False, False]).head()

,Movie_name,Movie_release,Actor_pairs,Movie_revenue,Movie_rating,Actor1,Actor2,Age_difference,Film_count_difference,Average_revenue_difference,First_film,First_film_for_one,Number_of_films_together,Same_genre,Genre,Rank,Ranking_ratio
396885,Terror in the Aisles,1984.0,"(Dean Martin, Jerry Lewis)",10004817.0,6.3,Dean Martin,Jerry Lewis,9.0,4,1.515564e+07,False,False,15,True,Comedy,336575,0.390451
294270,Pardners,1956.0,"(Dean Martin, Jerry Lewis)",3600000.0,6.1,Dean Martin,Jerry Lewis,9.0,0,0.000000e+00,False,False,13,True,Comedy,383695,0.305115
34036,Artists and Models,1955.0,"(Dean Martin, Jerry Lewis)",3800000.0,6.5,Dean Martin,Jerry Lewis,9.0,0,0.000000e+00,False,False,11,True,Comedy,291571,0.471955
549956,You're Never Too Young,1955.0,"(Dean Martin, Jerry Lewis)",3400000.0,6.5,Dean Martin,Jerry Lewis,9.0,0,0.000000e+00,False,False,11,True,Comedy,291805,0.471531
171113,Grown Ups,2010.0,"(Adam Sandler, Rob Schneider)",271430189.0,6.0,Adam Sandler,Rob Schneider,3.0,3,7.400058e+06,False,False,11,False,None,388990,0.295525


In [89]:
avg_rating_by_nb_films_together = df_pairs.groupby('Number_of_films_together')['Movie_rating'].mean()
fig = avg_rating_by_nb_films_together.plot(kind='bar')

fig.update_layout(
    title="Average rating by number of films together",
    xaxis_title="Number of films together",
    yaxis_title="Average rating",
    showlegend=False,
    xaxis={'tickmode':'linear'},
    yaxis_range=[1, 10],
)
fig.show()

In [90]:
model = smf.ols(formula='Movie_rating ~ Number_of_films_together', data=df_pairs)
model_fit = model.fit()
print(model_fit.summary())

                            OLS Regression Results                            
Dep. Variable:           Movie_rating   R-squared:                       0.001
Model:                            OLS   Adj. R-squared:                  0.001
Method:                 Least Squares   F-statistic:                     373.6
Date:                Fri, 22 Dec 2023   Prob (F-statistic):           3.26e-83
Time:                        17:54:12   Log-Likelihood:            -7.6697e+05
No. Observations:              552170   AIC:                         1.534e+06
Df Residuals:                  552168   BIC:                         1.534e+06
Df Model:                           1                                         
Covariance Type:            nonrobust                                         
                               coef    std err          t      P>|t|      [0.025      0.975]
--------------------------------------------------------------------------------------------
Intercept               

# Answer :

As we can observe in the plot, the average rating doesn't seem to depend on the frequency of actors costarring. This is confirmed by the test, where R_squared is very low, and thus ranking ratio doesn't depend on the frequency of actors costarring.